# Restaurant Data Import 

This notebook:
1. Loads restaurant data from the fz dataset
2. Imports clean data into a **constraint graph** 
3. Creates an **instance graph** 


**Workflow:**
- Environment setup → Load data → Import to Neo4j → Validate

## Step 1: Environment Configuration

Load Neo4j credentials and database names from `.env` file.
Never commit secrets — use `.env` (gitignored) for local credentials.


In [8]:
# --- Env loader & sanitiser ---

import os
import pathlib
from dotenv import load_dotenv
# .env path 
env_path = pathlib.Path.cwd() / ".env"
load_dotenv(dotenv_path=env_path, override=True)

def _strip_quotes(v):
    if v is None:
        return None
    return v.strip().strip('"').strip("'")

# sanitized env values
URI = _strip_quotes(os.getenv("NEO4J_URI"))
USERNAME = _strip_quotes(os.getenv("NEO4J_USERNAME"))
PASSWORD = _strip_quotes(os.getenv("NEO4J_PASSWORD"))

missing = [k for k,v in (("NEO4J_URI",URI),("NEO4J_USERNAME",USERNAME),("NEO4J_PASSWORD",PASSWORD)) if not v]
if missing:
    raise RuntimeError(f"Missing env vars: {missing}. Edit .env or export env vars and re-run this cell.")

# printing safe info only
print(f"Loaded Neo4j URI: {URI!r}, username: {USERNAME!r} (password hidden)")
AUTH = (USERNAME, PASSWORD)

Loaded Neo4j URI: 'neo4j://127.0.0.1:7687', username: 'neo4j' (password hidden)


## Step 2: Configure Databases & Parameters

Set constraint and instance database names, fraud count.
All values can be overridden via `.env` file.

In [11]:
# --- DB names, hyperparameters, and defaults ---
CONSTRAINT_DB = _strip_quotes(os.getenv("NEO4J_CONSTRAINT_DB")) or "test"
INSTANCE_DB   = _strip_quotes(os.getenv("NEO4J_INSTANCE_DB")) or "test-instance-graph"

from easydict import EasyDict as edict
hypp = edict()

print("Constraint DB:", CONSTRAINT_DB)
print("Instance DB:", INSTANCE_DB)


Constraint DB: restaurant-constraint
Instance DB: restaurant-instance


## Step 3: Locate Data Files

Auto-detects the latest cleaned restaurant and similarity files based on their timestampss from `datasets/temp/`.
These are generated by `restaurant_data_cleaning.ipynb`.

In [13]:
# --- Data paths (latest by timestamp) ---
from pathlib import Path
import re
from datetime import datetime

def latest_by_timestamp(folder, pattern, name_re, dt_fmt="%Y%m%d-%H%M%S"):
    folder = Path(folder)
    candidates = list(folder.glob(pattern))
    if not candidates:
        return None
    ts_files = []
    rx = re.compile(name_re)
    for p in candidates:
        m = rx.search(p.name)
        if m:
            try:
                ts = datetime.strptime(m.group(1), dt_fmt)
                ts_files.append((ts, p))
            except Exception:
                pass
    if ts_files:
        return str(max(ts_files, key=lambda t_p: t_p[0])[1])
    return str(max(candidates, key=lambda p: p.stat().st_mtime))

base = Path("datasets") / "temp"
RESTAURANTS_PATH = latest_by_timestamp(base, "restaurants_*.txt", r"restaurants_(\d{8}-\d{6})\.txt")
SIMILARITIES_PATH = latest_by_timestamp(base, "restaurant_similarities_*.txt", r"restaurant_similarities_(\d{8}-\d{6})\.txt")

print("Using RESTAURANTS_PATH =", RESTAURANTS_PATH)
print("Using SIMILARITIES_PATH =", SIMILARITIES_PATH)

Using RESTAURANTS_PATH = datasets\temp\restaurants_20260107-125146.txt
Using SIMILARITIES_PATH = datasets\temp\restaurant_similarities_20260107-125146.txt


## Step 4: Load Data into Memory

Read the cleaned restaurant nodes and similarity edges from the detected files into Python lists. 
This prepares the data for bulk import into Neo4j.

In [15]:
# --- Load restaurant nodes and similarity edges into memory
# Read restaurants file
restaurants = []
with open(RESTAURANTS_PATH, encoding="utf-8") as f:
    next(f)  # skip header
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) >= 5:
            # id, name, area_code, addr, city
            restaurants.append({
                "id": parts[0],
                "name": parts[1],
                "area_code": parts[2],
                "addr": parts[3],
                "city": parts[4]
            })

# Read similarity edges (expects lines like "(id1,id2)")
similarities = []
with open(SIMILARITIES_PATH, "r", encoding="utf-8") as file:
    for line in file:
        clean_line = line.strip().lstrip("(").rstrip(")")
        parts = [p.strip() for p in clean_line.split(",")]
        if len(parts) == 2:
            similarities.append((parts[0], parts[1]))

print(f"Loaded {len(restaurants)} restaurants and {len(similarities)} similarity edges.")
print("Sample restaurant:", restaurants[0])
print("Sample similarity edge:", similarities[0])

Loaded 864 restaurants and 5307 similarity edges.
Sample restaurant: {'id': '0', 'name': "arnie morton's of chicago", 'area_code': '310', 'addr': ' "435 s. la cienega blv."', 'city': ' "los angeles"'}
Sample similarity edge: ('0', '1')


## Step 5: Define Neo4j Operations

Helper functions:
- `clear_database()` — remove all nodes/relationships
- `setup_database()` — create uniqueness constraints
- `import_data()` — bulk insert restaurant nodes and similarity edges

In [20]:
# --- Neo4j helper functions and import workflow ---
from neo4j import GraphDatabase

def clear_database(driver, database):
    driver.execute_query("MATCH (n) DETACH DELETE n", database_=database)
    print(f"Database '{database}' cleared.")

def setup_database(driver, database):
    # Enforce unique restaurant id 
    driver.execute_query("""
        CREATE CONSTRAINT restaurant_id_unique IF NOT EXISTS
        FOR (r:Restaurant) REQUIRE r.id IS UNIQUE
    """, database_=database)
    print(f"Constraint ensured on '{database}'.")

def import_data(driver, restaurants_list, similarities_list, database):
    # Bulk create restaurant nodes
    driver.execute_query("""
        UNWIND $restaurants AS r
        MERGE (n:Restaurant {id: r.id})
        SET n.name = r.name, n.area_code = r.area_code, n.addr = r.addr, n.city = r.city
    """, restaurants=restaurants_list, database_=database)
    # Bulk create similarity edges
    driver.execute_query("""
        UNWIND $pairs AS pair
        MATCH (a:Restaurant {id: pair[0]})
        MATCH (b:Restaurant {id: pair[1]})
        MERGE (a)-[:SIMILAR]->(b)
    """, pairs=similarities_list, database_=database)
    print(f"Imported data into '{database}' (restaurants: {len(restaurants_list)}, pairs: {len(similarities_list)})")

def visualize_constraint_graph(driver, database):
    # Add self-loop CONSTRAINT edges so the area-code rule is visible in Neo4j Browser.
    driver.execute_query(
        "MATCH (:Restaurant)-[rel:CONSTRAINT]->() DELETE rel",
        database_=database,
    )
    driver.execute_query(
        "MATCH (r:Restaurant) MERGE (r)-[:CONSTRAINT]->(r)",
        database_=database,
    )
    print(f"Constraint self-loops refreshed on '{database}' (visualization only).")

In [21]:
# --- Verify connectivity and run import workflow ---

# Test connection
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Connected to Neo4j database successfully.")

# Run import: constraint DB (canonical)
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    clear_database(driver, CONSTRAINT_DB)
    setup_database(driver, CONSTRAINT_DB)
    import_data(driver, restaurants, similarities, CONSTRAINT_DB)
    visualize_constraint_graph(driver, CONSTRAINT_DB)  # optional visualization

# Run import: instance DB (sandbox)
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    clear_database(driver, INSTANCE_DB)
    # do NOT create the unique constraint on instance DB (to allow perturbations)
    import_data(driver, restaurants, similarities, INSTANCE_DB)

print("\nImport workflow complete.")

Connected to Neo4j database successfully.
Database 'restaurant-constraint' cleared.
Constraint ensured on 'restaurant-constraint'.
Imported data into 'restaurant-constraint' (restaurants: 864, pairs: 5307)
Constraint self-loops refreshed on 'restaurant-constraint' (visualization only).
Database 'restaurant-instance' cleared.
Imported data into 'restaurant-instance' (restaurants: 864, pairs: 5307)

Import workflow complete.


## Validation & Next Steps

- Constraint graph: clean, deduplicated author data with co-author relationships
- Instance graph: same structure with `hypp.fraud_number` swapped identities labeled `:Fraudulent`

**To verify in Neo4j Browser:**
```cypher
MATCH (a:Author) RETURN count(a) AS total_authors;
MATCH (a:Author:Fraudulent) RETURN count(a) AS swapped_authors;
MATCH (a:Author)-[:CO_AUTHOR]->(b:Author) RETURN count(*) AS co_author_edges LIMIT 1;